## Time-series plot of accumulated rainfall following the vortex

### Calculate area-averaged accumulated precipitation

In [ ]:
def calc_area_ave_acc_prcp(prcp, dataset, bv_lat, bv_lon, r0=3.0, plt_prcp=True):
    """
    calculate area-averaged, accumulated precipitation following a storm, within a specified radius 

    Args:
      prcp (Xarray DataArray): multi-dimensional data array 
      dataset (string): dataset to analyse (e.g. 4p4, N768, GPM) 
      bv_lat (Pandas DataFrame): vortex centre latitude from ERA-I tracking 
      bv_lon (Pandas DataFrame): vortex centre longitude from ERA-I tracking 

    Kwargs:
      r0 (float): radius for calculation (degrees) 
      plt_prcp (bool): plot precipitation as well as calculating acc precip following the storm 
    """

    # read in data and split into 12-h chunks 
    if dataset == '4p4':
        prcp = prcp * 3600. 
        prcp = prcp.resample(time="12H").sum().sel(time = slice('2018-10-21T12', 
                                                                '2018-10-26T12') )
    elif dataset == 'gpm':
        prcp = prcp[::2,:,:].resample(time="12H").sum().sel(time = slice('2018-10-21T12',
                                                                         '2018-10-26T00') )
        prcp = prcp.transpose('time','latitude','longitude')

    # initialise array to hold values of accumulated precipitation
    prcp_arr = xr.DataArray(np.ones(prcp.time.shape[0]),
                            dims=["time"],
                            coords={
                                "time": prcp.time
                                },
                            )

    # extract elements from (6-h) BV centre position arrays
    t_ind = np.arange(0, prcp.time.shape[0])
    if dataset == 'n768':
        xi = [4,6,8,10,12,14,16,18,20]
    else: 
        xi = [2,4,6,8,10,12,14,16,18,20]

    # define plot limits 
    n0 = np.rint(prcp.longitude[0].data); n1 = np.rint(prcp.longitude[-1].data)
    t0 = np.rint(prcp.latitude[0].data); t1 = np.rint(prcp.latitude[-1].data)
            
    # loop over times
    for i, it in enumerate(t_ind):

        # counter variable 
        xii = xi[i]

        if plt_prcp:

            # plot precipitation rate 
            fig = plt.figure(figsize=[9,6])
            ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0))

            # subset data 
            if dataset == 'n768':
                Levels=[0.1, 0.2, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 48.0, 64.0]
            else:
                Levels=[1.0, 2.0, 4.0, 8.0, 16.0, 24.0, 32.0, 48.0, 64.0, 96.0, 128.0]
            prcp = prcp.sel(longitude=slice(n0+0.5,n1-0.5), latitude=slice(t0+0.5,t1-0.5) )
            prcp[it,:,:].plot.contourf(ax=ax, levels=Levels, extend='max', 
                                       transform=ccrs.PlateCarree(),
                                       cbar_kwargs={'label': 'mm'},
                                       cmap=cmocean.cm.haline_r)

            # coastlines, tickmarks, etc
            ax.add_feature(cfeature.COASTLINE.with_scale('50m'),linewidth=0.75)
        
            # date string 
            dstr = prcp.time.dt.strftime("%Y%m%dT%H").values

            # set up plot 
            fili = './prcp_{1}_{0}.png'.format(dstr[it],dataset)

            # define plot limits and set up tickmarks 
            xint = 5; yint = 3
            ns = np.rint(prcp.longitude[0].data); nf = np.rint(prcp.longitude[-1].data)
            ts = np.rint(prcp.latitude[0].data); tf = np.rint(prcp.latitude[-1].data)
            ax.set_xticks(np.arange(ns, nf+1, xint))
            ax.set_xticklabels(np.arange(ns, nf+1, xint))
            ax.set_yticks(np.arange(ts, tf+1, yint))
            ax.set_yticklabels(np.arange(ts, tf+1, yint))

            # add gridlines 
            plt.gca().gridlines(color='grey', linestyle='--', linewidth=0.5)
            plt.title('')

            # overlay BV box position 
            ax.add_patch( Rectangle( (bv_lon[xii]-r0, bv_lat[xii]-r0),
                                     2*r0, 2*r0, linewidth=2,
                                     facecolor='none', edgecolor='k') )

            # save figure and continue 
            fig.savefig(fili,dpi=200)

        # calculate area-averaged, accumulated precipitation 
        pr = prcp[it,:,:]
        pr = pr.loc[bv_lat[xii]-r0:bv_lat[xii]+r0,
                    bv_lon[xii]-r0:bv_lon[xii]+r0].mean()

        prcp_arr[i] = pr.data

    return prcp_arr

### Process area-averaged accumulated precipitation before plotting

In [ ]:
def calc_acc_prcp_time_series(vortex_path, prcp_gpm, prcp_4p4_metum, prcp_n768_metum, vortex_box_radius):

    bv_lat, bv_lon, bv_time = extract_vortex_info(vortex_path)

    acc_prcp_gpm = calc_area_ave_acc_prcp(prcp_gpm, 'gpm', bv_lat, bv_lon, r0=vortex_box_radius, plt_prcp=False)
    acc_prcp_4p4 = calc_area_ave_acc_prcp(prcp_4p4_metum,'4p4',bv_lat,bv_lon,r0=vortex_box_radius,plt_prcp=False)
    acc_prcp_n768 = calc_area_ave_acc_prcp(prcp_n768_metum,'n768',bv_lat,bv_lon,r0=vortex_box_radius,plt_prcp=False)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(bv_time, np.zeros(21), color='k', alpha=0.0)
    ax.plot(bv_time[2:21:2], acc_prcp_4p4, color='k', label='4.4 km MetUM')
    ax.plot(bv_time[4:21:2], acc_prcp_n768, color='b', label='Global MetUM')
    ax.plot(bv_time[2:21:2], acc_prcp_gpm, color='r', label='GPM-IMERG')
    ax.set(xlabel='Time', ylabel='Accumulated rainfall (mm)',
           title='Time-series of area-averaged, accumulated rainfall')
    ax.grid(True)
    ax.legend(loc='upper left')

    OUT_PATH = './acc_prcp_oct2018_{0}deg.png'.format(vortex_box_radius)
    fig.savefig(OUT_PATH, dpi=400)

    return fig

### Plotting function

In [ ]:
def plot_prcp_time_series(bounds, vortex_path, vortex_box_radius):

    METUM_4p4_PATH = '/nobackup/earshar/borneo/20181021T1200Z_SEA4_km4p4_ra1tld_pverb.pp'
    pcubes = iris.load(METUM_4p4_PATH)
    prcp_4p4_metum = xr.DataArray.from_iris(pcubes.extract('stratiform_rainfall_flux')[1])
    prcp_4p4_metum = prcp_4p4_metum.sel(longitude=slice(bounds[0], bounds[1]), latitude=slice(bounds[2], bounds[3]))

    GPM_PATH = '/nobackup/earshar/borneo/GPMHH_201810.nc'
    gpm_data = xr.open_dataset(GPM_PATH).sel(lon=slice(bounds[0], bounds[1]), lat=slice(bounds[2], bounds[3]))
    prcp_gpm = gpm_data.precipitationCal

    Tp = [24, 36, 48, 60, 72, 84, 96, 108, 120]
    prcp_n768_metum_dims = [len(Tp), prcp_4p4_metum.latitude.shape[0], prcp_4p4_metum.longitude.shape[0]]
    prcp_gpm_resample_12h = prcp_gpm.resample(time="12H").sum().sel(time=slice('2018-10-22T12',
                                                                               '2018-10-26T12'))

    prcp_n768_metum = xr.DataArray(np.ones(prcp_n768_metum_dims),
                           dims=["time", "latitude", "longitude"],
                           coords={
                               "time": prcp_gpm_resample_12h.time,
                               "latitude": prcp_4p4_metum.latitude,
                               "longitude": prcp_4p4_metum.longitude,
                           },
                           )

    for i, t in enumerate(Tp):
        METUM_N768_PATH = '/nobackup/earshar/borneo/case_20181021T1200Z_N768/nc/umglaa_pa{0:03d}.nc'.format(t - 12)
        # METUM_N768_PATH = '/nobackup/earshar/borneo/case_20181021T1200Z_N768_v2/umglaa_pa{0:03d}.nc'.format(t - 12)
        data_n768_metum = xr.open_dataset(METUM_N768_PATH)
        data_n768_metum=data_n768_metum["tot_precip"].squeeze('t_1').squeeze('surface').sel(longitude=slice(bounds[0],
                                                                                                            bounds[1]),
                                                                                            latitude=slice(bounds[2],
                                                                                                           bounds[3]))
        # interpolate N768 MetUM data onto 4p4 MetUM grid
        prcp_n768_metum[i, :, :] = data_n768_metum.interp(longitude=prcp_4p4_metum["longitude"],
                                                          latitude=prcp_4p4_metum["latitude"],
                                                          method="linear")

    # also interpolate GPM data onto 4p4 MetUM grid
    prcp_gpm = prcp_gpm.interp(lon=prcp_4p4_metum["longitude"],
                               lat=prcp_4p4_metum["latitude"],
                               method="linear")

    fig = calc_acc_prcp_time_series(vortex_path, prcp_gpm, prcp_4p4_metum, prcp_n768_metum, vortex_box_radius)

    return fig